In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import sys
sys.path.append('..')

In [3]:
from scraper import Scraper

In [4]:
with open("hypos.txt") as f:
    hypos = [line.strip() for line in f]

with open("refs.txt") as f:
    refs = [line.strip() for line in f]

In [5]:
refs

['https://baoantelecom.com/vi/tin/dien-tu-vien-thong',
 'https://www.hotcourses.vn/study-abroad-info/study-guides/nganh-dien-tu-vien-thong/',
 'https://blog.topcv.vn/co-nen-hoc-nganh-dien-tu-vien-thong/']

In [6]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"

In [7]:
refs = Scraper(refs, user_agent).run()

In [8]:
refs

[{'url': 'https://baoantelecom.com/vi/tin/dien-tu-vien-thong',
  'raw_content': 'Tìm hiểu kiến thức học và cơ hội việc làm ngành Điện tử viễn thông\nMỤC LỤC\n1. Ngành điện tử viễn thông là gì? Và ứng dụng trong đời sống\n2. Ngành điện tử viễn thông học những kiến thức gì?\n3. Định hướng công việc ngành Điện tử viễn thông? Cơ hội việc làm có rộng mở?\n4. Cần trang bị gì để học và làm việc tốt ngành điện tử viễn thông\n5. Các trường đào tạo ngành điện tử viễn thông\nNgành điện tử viễn thông là gì? Và ứng dụng trong đời sống\nĐiện tử viễn thông là ngành sử dụng những công nghệ tiên tiến để tạo ra các thiết bị giúp cho việc liên lạc, truy xuất thông tin của cá nhân hoặc tổ chức, điện tử viễn thông được ứng dụng cao ở nhiều lĩnh vực trong cuộc sống.\n•\xa0\xa0 \xa0Lĩnh vực mạng, viễn thông: các thiết bị truyền tin trên toàn cầu như hệ thống truyền dẫn: cáp quang, vệ tin, hệ thống truyền tin không dây (vi ba) v.v... Thành tựu mới đây nhất là xây dựng hệ thống 5g và các thiết bị hệ thống hôi 

In [9]:
len(refs)

3

In [10]:
refs = [item['raw_content'] for item in refs]

In [11]:
refs

['Tìm hiểu kiến thức học và cơ hội việc làm ngành Điện tử viễn thông\nMỤC LỤC\n1. Ngành điện tử viễn thông là gì? Và ứng dụng trong đời sống\n2. Ngành điện tử viễn thông học những kiến thức gì?\n3. Định hướng công việc ngành Điện tử viễn thông? Cơ hội việc làm có rộng mở?\n4. Cần trang bị gì để học và làm việc tốt ngành điện tử viễn thông\n5. Các trường đào tạo ngành điện tử viễn thông\nNgành điện tử viễn thông là gì? Và ứng dụng trong đời sống\nĐiện tử viễn thông là ngành sử dụng những công nghệ tiên tiến để tạo ra các thiết bị giúp cho việc liên lạc, truy xuất thông tin của cá nhân hoặc tổ chức, điện tử viễn thông được ứng dụng cao ở nhiều lĩnh vực trong cuộc sống.\n•\xa0\xa0 \xa0Lĩnh vực mạng, viễn thông: các thiết bị truyền tin trên toàn cầu như hệ thống truyền dẫn: cáp quang, vệ tin, hệ thống truyền tin không dây (vi ba) v.v... Thành tựu mới đây nhất là xây dựng hệ thống 5g và các thiết bị hệ thống hôi nghị truyền hình\n•\xa0\xa0 \xa0Lĩnh vực định vị dẫn đường: là một lĩnh vực đặc

In [12]:
from unidecode import unidecode
import re

def normalize_text(text):
    """
    Normalize Vietnamese text by converting it to its ASCII representation and removing accents.
    
    Args:
    text (str): Input Vietnamese text to be normalized.
    
    Returns:
    str: Normalized text with accents removed.
    """
    normalized_text = unidecode(text)
    normalized_text = re.sub(r'\s+', ' ', normalized_text).strip()
    return normalized_text


In [13]:
hypos = [' '.join(normalize_text(hypo) for hypo in hypos)]

In [14]:
hypos

['# Co Hoi Nghe Nghiep Cho Sinh Vien Nganh Dien Tu - Vien Thong Cua Dai Hoc Bach Khoa Ha Noi Sau Khi Tot Nghiep  Nganh Dien tu - Vien thong da va dang tro thanh mot trong nhung nganh hoc quan trong va khong the thieu trong boi canh cuoc cach mang cong nghe 4.0 dang dien ra manh me tren toan cau. Doi voi sinh vien Dai hoc Bach khoa Ha Noi, mot trong nhung truong dai hoc hang dau Viet Nam, co hoi nghe nghiep sau khi tot nghiep nganh nay la vo cung rong mo va da dang.  ## Tong Quan ve Nganh Dien tu - Vien Thong  Nganh Dien tu - Vien thong bao gom viec nghien cuu, thiet ke, phat trien va ung dung cac cong nghe lien quan den dien tu va vien thong. Sinh vien nganh nay duoc dao tao de tro thanh nhung ky su co kha nang lam viec trong moi truong cong nghe cao, dap ung nhu cau ngay cang tang ve viec thiet ke va van hanh cac he thong thong tin va truyen thong hien dai (TopCV, n.d.).  ## Co Hoi Nghe Nghiep Sau Khi Tot Nghiep  ### Cac Vi Tri Cong Viec Tiem Nang  Sinh vien tot nghiep nganh Dien tu -

In [15]:
refs = [normalize_text(ref) for ref in refs]

In [16]:
import textwrap

def wraptext(documents, width=120):
    for idx, doc in enumerate(documents, start=1):
        print(f"Document {idx}:")
        wrapped_lines = textwrap.wrap(doc, width=width)
        for line in wrapped_lines:
            print(line)
        print("-" * width)

In [17]:
print(wraptext(refs))

Document 1:
Tim hieu kien thuc hoc va co hoi viec lam nganh Dien tu vien thong MUC LUC 1. Nganh dien tu vien thong la gi? Va ung
dung trong doi song 2. Nganh dien tu vien thong hoc nhung kien thuc gi? 3. Dinh huong cong viec nganh Dien tu vien
thong? Co hoi viec lam co rong mo? 4. Can trang bi gi de hoc va lam viec tot nganh dien tu vien thong 5. Cac truong dao
tao nganh dien tu vien thong Nganh dien tu vien thong la gi? Va ung dung trong doi song Dien tu vien thong la nganh su
dung nhung cong nghe tien tien de tao ra cac thiet bi giup cho viec lien lac, truy xuat thong tin cua ca nhan hoac to
chuc, dien tu vien thong duoc ung dung cao o nhieu linh vuc trong cuoc song. * Linh vuc mang, vien thong: cac thiet bi
truyen tin tren toan cau nhu he thong truyen dan: cap quang, ve tin, he thong truyen tin khong day (vi ba) v.v... Thanh
tuu moi day nhat la xay dung he thong 5g va cac thiet bi he thong hoi nghi truyen hinh * Linh vuc dinh vi dan duong: la
mot linh vuc dac biet quan trong nganh H

In [18]:
len(hypos), len(refs)

(1, 3)

### Using BERTScore for Evaluation

1. BERTScore is used to measure textual similarity between candidate texts and reference texts. It considers not only exact word matches but also the overall meaning, fluency, and order of the ouput.

2. BERTScore: Precision, Recall, F1
    * Precision measures how well the candidate texts avoid introducing irrelevant content.
    * Recall measures how well the candidate texts avoid omitting relevant content.
    * F1 = 2 x (P x R)/(P + R) 

In [19]:
from bert_score import score

/media/sanslab/Data/stevehoang/rag_research/vigpt-researcher/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# Hide the loading messages
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

In [21]:
def bert_score(hypos, refs, lang="vi"):
    bert_scores = []
    for ref in refs:
        scores = score(hypos, [ref], lang=lang, verbose=False)
        bert_scores.append(scores)
    return bert_scores

In [22]:
bert_scores = bert_score(hypos, refs)

In [23]:
import torch

precision = torch.mean(torch.stack([bert_score[0] for bert_score in bert_scores]))
recall = torch.mean(torch.stack([bert_score[1] for bert_score in bert_scores]))
f1 = torch.mean(torch.stack([bert_score[2] for bert_score in bert_scores]))

In [24]:
print("BERT score:")
print(f"P: {precision.item():.2f}, R: {recall.item():.2f}, F1: {f1.item():.2f}")

BERT score:
P: 0.74, R: 0.76, F1: 0.75


### Using ROUGE score for Evaluation

In [25]:
from rouge_score import rouge_scorer

def rouge_score(hypos, refs):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = [scorer.score(ref, hypo) for ref, hypo in zip(refs, hypos)]
    return scores

rouge = rouge_score(hypos, refs)

In [26]:
for i, score in enumerate(rouge):
    print("ROUGE scores:")
    print(f"ROUGE-1: P = {score['rouge1'].precision:.2f}, R = {score['rouge1'].recall:.2f}, F1 = {score['rouge1'].fmeasure:.2f}")
    print(f"ROUGE-2: P = {score['rouge2'].precision:.2f}, R = {score['rouge2'].recall:.2f}, F1 = {score['rouge2'].fmeasure:.2f}")

ROUGE scores:
ROUGE-1: P = 0.82, R = 0.44, F1 = 0.57
ROUGE-2: P = 0.42, R = 0.23, F1 = 0.29


### RESULTS

| METRIC   | Precision | Recall | F1 |
|---------|----------|-------|-------|
| BERTScore| | | |
| ROUGE-1| | | |
| ROUGE-2| | | |

* BERTScore Average: Precision = ?, Recall = ?, F1 = ?
* ROUGE-1 Average: Precision = ?, Recall = ?, F1 = ?
* ROUGE-2 Average: Precision = ?, Recall = ?, F1 = ?

### Total Time Executions

|Time (s) | T1 | T2 |
|----|----|----|
|    | 100.123 | 123.123 |

### Review
1. Summarization is a complex task, even advanced models can struggle to accurately capture all the important information from a source text.

2. BERTScore or ROUGE that these metrics focus on things like including keywords from the source text, so that do not perfectly reflect human judgement of a good summary.

3. The response of chatbot from many different sources.


### HUMAN EVALUATION IS THE BEST WAY TO EVALUATE THE QUALITY OF A SUMMARY